# LSTM Autoencoder for Audio

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed

In [ ]:
model = Sequential()
model.add(LSTM(..., input_shape = (...)))
model.add(LSTM(..., return_sequences = True))
model.add(TimeDistributed(Dense(...)))

In [ ]:
model = Sequential()
model.add(LSTM(100, activation = 'relu', input_shape = (n_in, 1)))
model.add(RepeatVector(n_out))
model.add(LSTM(100, activation = 'relu', return_sequences = True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer = 'adam', loss = 'mse')
model.summary()